# By Himanshu Shukla
#### Please feel free to ask for anything, if you need some different integrations, i used pure libraries here
#### email himanshushukla.shukla3@gmail.com
#### Environment: Windows 10, python 3.11.5, PyTorch version: 2.2.1+cu121 | CUDA version: 12.1

In [ ]:
!pip install diffusers --upgrade > log.txt

## In addition make sure to install transformers, safetensors, accelerate as well as the invisible watermark:

In [ ]:
!pip install invisible_watermark transformers accelerate safetensors > log.txt


# Model from hugging face

In [ ]:
from diffusers import DiffusionPipeline
import torch

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")



# Define how many steps and what % of steps to be run on each experts (80/20) here

In [ ]:
n_steps = 1000
high_noise_frac = 0.8

# run both experts

In [ ]:
# providing Prompt

prompt = input(" Enter prompt ")

image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

# Showing Image
image

In [ ]:
image.size

# Saving the image locally, then We'll give to GAN for scaling the resolution

In [ ]:
image.save("generated_image.png")

In [ ]:
!pip install pillow > log.txt
!pip install -qq super-image > log.txt


In [ ]:
from super_image import EdsrModel, ImageLoader
from PIL import Image
import requests
from IPython.display import display

# Load the model
model = EdsrModel.from_pretrained('eugenesiow/edsr-base', scale=2)

# Load the image using PIL and then process it using ImageLoader
image_path = "generated_image.png"
image = Image.open(image_path)
inputs = ImageLoader.load_image(image)

# Generate predictions
preds = model(inputs)

# Save the upscaled image and comparison
ImageLoader.save_image(preds, './scaled_2x.png')
ImageLoader.save_compare(inputs, preds, './scaled_2x_compare.png')



In [ ]:
scaled_image = Image.open("./scaled_2x.png")



In [ ]:
display(scaled_image)

In [ ]:
# Size of image in pixels
scaled_image.size
